In [ ]:
import numpy as np
import math
import tensorflow as tf
import pandas as pd
from util import R
from sklearn.metrics.pairwise import cosine_similarity


In [ ]:
# R is the radius of cell nuclear
chr_name_list=["chr1","chr2","chr3","chr4","chr5","chr6","chr7","chr8","chr9","chr10","chr11","chr12","chr13","chr14","chr15","chr16","chr17","chr18","chr19"]

V_nucl = 4/3*math.pi*(R**3)
cut_thick = 0.2 #200nm
dic_chr_size={"chr1":195471971,"chr2":182113224,"chr3":160039680,"chr4":156508116,"chr5":151834684,"chr6":149736546,"chr7":145441459,"chr8":129401213,"chr9":124595110,"chr10":130694993,"chr11":122082543,"chr12":120129022,"chr13":120421639,"chr14":124902244,"chr15":104043685,"chr16":98207768,"chr17":94987271,"chr18":90702639,"chr19":61431566,"chrX":171031299}
genome_length = np.sum(dic_chr_size.values())
all_v_necl = [float(dic_chr_size[i])/genome_length*(V_nucl/2) for i in dic_chr_size]


In [ ]:
dir_npfile = "./seqGAM_data/merge_219_414_capturerate_eachNP_10000bp_withstrain.csv"
npfile = pd.read_csv(dir_npfile)
#um
npfile['capture_s'] = npfile['capture_rate']*V_nucl/(2*cut_thick) 
npfile['distance_to_nucl_center'] = np.abs(npfile['zaxis'] - R)
npfile.head()



In [ ]:

mean_logits = np.load("./seqGAM_data/mean_logits_0.8381.npz.npy")
# npzfile
all_theta_twice_matrix = cosine_similarity(mean_logits)
# all_theta_multi.shape #(40, 40)
all_theta_twice_number = np.arccos(all_theta_twice_matrix)
# theta_twice = np.nanmin(all_theta_number[np.where(all_theta_number!=0)])
theta_twice = np.sort(all_theta_twice_number[np.where(all_theta_twice_number!=0)])
# print(theta_twice[0:10])

# theta_multi = 5.7715875e-01
theta_multi = 5.7715875e-01

c = R/2
theta = theta_multi/2
a = b = R/2*math.sin(theta)
print([a,b,c])


In [ ]:
import math
import numpy as np
from scipy import integrate
import matplotlib.pyplot as plt

def area_of_shape(a, b, c, D, theta):
    D_to_center = c*np.sin(theta) - D
#     x = D * np.tan(theta) / c
    x = D_to_center * np.tan(theta) / c
    mask = np.abs(x) <= 1
    area_ellipse = np.zeros_like(x)
    area_triangle = np.zeros_like(x)
    a_p = a * np.sqrt(1 - x**2)
    b_p = b * np.sqrt(1 - x**2)
    area_ellipse[mask] = np.pi * a_p[mask] * b_p[mask]
#     print(x[mask].shape)
#     print(area_triangle.shape)
    area_triangle[mask] = np.pi * a * b * np.sqrt(1 - x[mask]**2) * np.sin(theta[mask])
    return area_ellipse + area_triangle

def grid_search_d_theta(a, b, c, D, areas_get):
#     D = np.random.uniform(0, c, 1000)
#     theta = np.random.uniform(0, math.pi/2, 10000)
    theta = np.random.uniform(0, math.pi, 100000)
    errors = np.abs(areas_get - area_of_shape(a, b, c, D, theta))
    return theta[np.argmin(errors)], errors[np.argmin(errors)]

def grid_search_d_theta(a, b, c, D, areas_get):
#     D = np.random.uniform(0, c, 1000)
    theta = np.random.uniform(0, math.pi, 10000)
    errors = np.abs(areas_get - area_of_shape(a, b, c, D, theta))
    return theta[np.argmin(errors)], errors[np.argmin(errors)]


def weighted_mse_loss(input_array, target_array, weight):
    return np.mean(weight * (input_array - target_array) ** 2)

def multi_weighted_mse_loss(input_array, target_array, weight):
    return np.mean(weight * (input_array - target_array) ** 2, axis = 1)

"""
disorder_list is the list without order, actually a set
disorder_list1: calculated angle by data
disorder_list2: calculated anlge by searched theta, phi
weight_list1 is the weight for element in disorder_list1
return the weighted loss
"""
def similarity_between_two_set(disorder_list1, disorder_list2, weight_list1):
    angle_array1 = np.array(sorted(disorder_list1))
    angle_array2 = np.array(sorted(disorder_list2))
    angle_weight_array1 = np.array([x for (y,x) in sorted(zip(disorder_list1,weight_list1))])
    return weighted_mse_loss(angle_array1,angle_array2, angle_weight_array1)

#the smaller the better
def multi_similarity_between_two_set(disorder_list1, disorder_list2, weight_list1):
    angle_array1 = np.sort(disorder_list1)
    angle_array2 = np.sort(disorder_list2,axis=1)
    angle_weight_array1 = np.array([x for (y,x) in sorted(zip(disorder_list1,weight_list1))])
    return multi_weighted_mse_loss(angle_array1,angle_array2, angle_weight_array1)

"""
theta_plane_here, phi_plane_here: both are float 
cone_axis_here: list like [x,y,z]
"""
# The incoming theta_plane, phi_plane are tensor.
# Calculate the angle between the coordinates of the given cone and the vertical line of the cutting plane, and subtract it from π/2 to get the angle between the main axis and the cutting plane.
def theta_plane_cone_axis(theta_plane_here, phi_plane_here, cone_axis_here):
    r = R
    x = r * np.cos(phi_plane_here) * np.sin(theta_plane_here)
    y = r * np.sin(phi_plane_here) * np.sin(theta_plane_here)
    z = r * np.cos(theta_plane_here)
    v_plane = [x, y, z]
    theta_vp_c = np.arccos( cosine_similarity([cone_axis_here], [v_plane]) ) #vertical plane with cone
    if theta_vp_c > math.pi/2:
        theta_vp_c = math.pi - theta_vp_c
    return math.pi/2 - theta_vp_c

def multi_theta_plane_cone_axis(theta_plane_here, phi_plane_here, cone_axis_here):
    r = 1
    x = r * np.cos(phi_plane_here) * np.sin(theta_plane_here)
    y = r * np.sin(phi_plane_here) * np.sin(theta_plane_here)
    z = r * np.cos(theta_plane_here)
    v_plane = [[x[i], y[i], z[i]] for i in range(len(x))]
    theta_vp_c = np.arccos( cosine_similarity([cone_axis_here], v_plane) ) #vertical plane with cone
    theta_vp_c_ = np.where((theta_vp_c > math.pi/2), math.pi - theta_vp_c, theta_vp_c)
    return math.pi/2 - theta_vp_c_

def get_all_cone_theta_plane_cone_axis(theta_plane_here, phi_plane_here, cone_axis):
    all_cone_theta_plane_cone_axis = []
    for cone_axis_here in cone_axis:
        angle_here = theta_plane_cone_axis(theta_plane_here, phi_plane_here, cone_axis_here)[0][0]
        all_cone_theta_plane_cone_axis.append( angle_here )
    return all_cone_theta_plane_cone_axis
    
def multi_get_all_cone_theta_plane_cone_axis(theta_plane_here, phi_plane_here, cone_axis):
    all_cone_theta_plane_cone_axis = []
    for cone_axis_here in cone_axis:
        angle_here = multi_theta_plane_cone_axis(theta_plane_here, phi_plane_here, cone_axis_here)[0]
        all_cone_theta_plane_cone_axis.append( angle_here )
    return np.transpose(all_cone_theta_plane_cone_axis) #return [theta_num, cone_num], [1000, 40]


In [ ]:
cell_chr_strain_2_angle = {}

for chr_explore in chr_name_list:
    for strain in [0,1]:
        area_list = np.array(list(npfile.loc[(npfile.cell_name==cell_name)&(npfile.chr_name==chr_explore)&(npfile.strain_index==strain),'capture_s']))
        distance_list = np.array(list(npfile.loc[(npfile.cell_name==cell_name)&(npfile.chr_name==chr_explore)&(npfile.strain_index==strain),'distance_to_nucl_center']))
        Ds = np.abs(R-np.abs(distance_list))
        areas = np.abs(area_list)
        all_angle = []
        all_error = []
        for i in range(len(Ds)):
            D = np.abs(Ds[i])
            A = areas[i]
            theta_cal,error_cal = grid_search_d_theta(a, b, c, D, A)
            all_angle.append(theta_cal)
            all_error.append(error_cal)

        print("===="+cell_name+'_'+str(strain)+'_'+chr_explore+"\nmean_theta %s, median_theta %s\ntheta %s\nerror %s"%(np.nanmean(all_angle), np.nanmedian(all_angle), all_angle, all_error))
        cell_chr_strain_2_angle[cell_name+'_'+str(strain)+'_'+chr_explore] = np.nanmean(all_angle)


In [ ]:
# Here N selects the top N chr with the largest average area (in the given cell)
to_see_chr_num = 3
# to_see_chr_num = 38

npfile_thiscell = npfile.loc[npfile.cell_name==cell_name].copy()

# area_list = np.array(list(npfile.loc[(npfile.cell_name==cell_name)&(npfile.chr_name==chr_explore)&(npfile.strain_index==strain),'capture_s']))
# distance_list = np.array(list(npfile.loc[(npfile.cell_name==cell_name)&(npfile.chr_name==chr_explore)&(npfile.strain_index==strain),'distance_to_nucl_center']))
# npfile_thiscell.groupby(['chr_name', 'strain_index'])[['capture_s']].agg(['mean']).sort_values()
npfile_thiscell_group = npfile_thiscell.groupby(['chr_name', 'strain_index'])[['capture_s']].agg(['mean'])
npfile_thiscell_group.columns = npfile_thiscell_group.columns.droplevel([0])
npfile_thiscell_group = npfile_thiscell_group.sort_values(by='mean', ascending=[False]).reset_index()
npfile_thiscell_group.head()

top_chr = list(npfile_thiscell_group.chr_name[0:to_see_chr_num])
top_strain = list(npfile_thiscell_group.strain_index[0:to_see_chr_num])
top_area = list(npfile_thiscell_group['mean'])[0:to_see_chr_num]

print('biggest area chr %s, strain %s, area %s'%(top_chr,top_strain, top_area))


In [ ]:
# 计算当只有N个cone的时候，切片的角度(用矩阵计算加速)
# 用搜索计算正确的切片角度
from itertools import combinations,permutations
import copy

cone_axis = copy.deepcopy(mean_logits)
caculated_theta = copy.deepcopy(cell_chr_strain_2_angle.values())

caculated_theta_38 = [cell_chr_strain_2_angle[cell_name+'_'+str(top_strain[i])+'_'+top_chr[i]] for i in range(to_see_chr_num)]
weight_list1_38 = [1/40.0 for i in range(to_see_chr_num)]

divide_num = 100
theta_x = np.linspace(0, math.pi, num = divide_num, endpoint = True)
phi_x = np.linspace(0, math.pi, num = divide_num, endpoint = True)
xa, xb = np.meshgrid(theta_x, phi_x)  
theta_x = xa.reshape(divide_num**2)
phi_x = xb.reshape(divide_num**2)
theta_here = theta_x
phi_here = phi_x
min_similarity = np.inf
final_posible_plot_x, final_posible_plot_y, final_posible_plot_z = theta_x, phi_x, []
final_posible_cone_axis = []

len_search = len(list(combinations(cone_axis,3)))

for idx_combine, posible_cone_axis in enumerate(list(combinations(cone_axis,to_see_chr_num))):
    posible_cone_axis = list(posible_cone_axis)
    all_cone_theta_plane_cone_axis_here = multi_get_all_cone_theta_plane_cone_axis(theta_here, phi_here, posible_cone_axis)
    simi_result = multi_similarity_between_two_set(caculated_theta_38, all_cone_theta_plane_cone_axis_here, weight_list1_38)
    if np.min(simi_result) < min_similarity:
        min_similarity = np.min(simi_result)
        final_posible_plot_z = simi_result
        final_posible_cone_axis = posible_cone_axis
        print("posible_cone_axis idx_combine %s"%(idx_combine))
        print([x for (y,x) in sorted(zip(simi_result,theta_here))][0:3])
        print([x for (y,x) in sorted(zip(simi_result,phi_here))][0:3])
        print(sorted(simi_result)[0:3])
    if idx_combine%10000==0:
        print("posible_cone_axis idx_combine %s of %s"%(idx_combine,len_search))



In [ ]:
# Determine the correspondence between candidates and cell_strain_chr

theta_now = [x for (y,x) in sorted(zip(final_posible_plot_z,theta_here))][0]
phi_now = [x for (y,x) in sorted(zip(final_posible_plot_z,phi_here))][0]


# all_cone_theta_plane_cone_axis_here = multi_get_all_cone_theta_plane_cone_axis(theta_now, phi_now, posible_cone_axis)
# Calculate the angle set between the selected cone set and the selected slice
all_cone_theta_plane_cone_axis_here = get_all_cone_theta_plane_cone_axis(theta_now, phi_now, final_posible_cone_axis)
# simi_result = similarity_between_two_set(caculated_theta_38, all_cone_theta_plane_cone_axis_here, weight_list1_38)

# Sorted the ground truth angle from smallest to largest, output the corresponding cone index
# sorted_ground_truth_cone_index = [x for (y,x) in sorted(zip(ground_truth_all_cone, range(len(ground_truth_all_cone))))] 

# The angle between the selected cone set and the slice is sorted from smallest to largest, and the index of the cone is output to get the [x,y,z] of the cone
sorted_candidates_cone_index = [x for (y,x) in sorted(zip(all_cone_theta_plane_cone_axis_here, range(len(all_cone_theta_plane_cone_axis_here))))] 
sorted_candidates_cone_xyz = [final_posible_cone_axis[i] for i in sorted_candidates_cone_index ]

# Arrange the calculated angle between the chr and the slice from smallest to largest, and output the name of the chr
sorted_chr_cone_index = [x for (y,x) in sorted(zip(caculated_theta_38, [cell_name+'_'+str(top_strain[i])+'_'+top_chr[i]  for i in range(to_see_chr_num)] ))] 

result_dic_chr_to_cone = dict(zip(sorted_chr_cone_index, sorted_candidates_cone_xyz))
result_dic_chr_to_cone


In [ ]:
np.save('./Sequential_CT_result/chr_to_cone_'+cell_name+'_top_'+str(len(result_dic_chr_to_cone))+'.npy', result_dic_chr_to_cone)
print('saved in: '+'./Sequential_CT_result/chr_to_cone_'+cell_name+'_top_'+str(len(result_dic_chr_to_cone))+'.npy')


In [ ]:
from mpl_toolkits.mplot3d import Axes3D, axes3d
import matplotlib.pyplot as plt
import numpy as np
sns.set_style("white")


result_plot_to_see = final_posible_cone_axis

fig = plt.figure(figsize=(6,6),dpi=300)
ax = fig.add_subplot(111, projection='3d')


x = [i[0] for i in result_plot_to_see]
y = [i[1] for i in result_plot_to_see]
z = [i[2] for i in result_plot_to_see]
c1 = [range(len(x))]

ax.scatter3D(x, y, z, marker='o',c='red')

ax.set_xlabel('X') 
ax.set_ylabel('Y') 
ax.set_zlabel('Z') 
ax.set_xlim(-R, R)
ax.set_ylim(-R, R)
ax.set_zlim(-R, R)


for key in result_dic_chr_to_cone:
    ax.text(result_dic_chr_to_cone[key][0], result_dic_chr_to_cone[key][1], result_dic_chr_to_cone[key][2], key.replace(cell_name, ''), color='red')
    
    
# plotting the nuclear
center = [0, 0, 0]
radius = R

u = np.linspace(0, 2 * np.pi, 100)
v = np.linspace(0, np.pi, 100)
x = radius * np.outer(np.cos(u), np.sin(v)) + center[0]
y = radius * np.outer(np.sin(u), np.sin(v)) + center[1]
z = radius * np.outer(np.ones(np.size(u)), np.cos(v)) + center[2]
ax.plot_surface(x, y, z, rstride=9, cstride=9, color='b',alpha = 0.1)

# ax.grid(True)
ax.grid(False)

viewa, viewb = 0, 30
ax.view_init(viewa, viewb)
plt.savefig('./Sequential_CT_result/relative_angle_3d_'+cell_name+'_top_'+str(len(result_dic_chr_to_cone))+'rangeR_'+str(viewa)+'_'+str(viewb)+'.pdf', dpi=300)
print('./Sequential_CT_result/relative_angle_3d_'+cell_name+'_top_'+str(len(result_dic_chr_to_cone))+'rangeR_'+str(viewa)+'_'+str(viewb)+'.pdf')

